In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
!pip install "stable-baselines3[extra]>=2.0.0a4"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  freeglut3 libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgles-dev libgles1 libglu1-mesa
  libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libice-dev libopengl-dev libsm-dev
  libxfont2 libxkbfile1 libxt-dev x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
Suggested packages:
  libice-doc libsm-doc libxt-doc
The following NEW packages will be installed:
  freeglut3 freeglut3-dev libegl-dev libfontenc1 libgl-dev libgl1-mesa-dev libgles-dev libgles1
  libglu1-mesa libglu1-mesa-dev libglvnd-core-dev libglvnd-dev libglx-dev libice-dev libopengl-dev
  libsm-dev libxfont2 libxkbfile1 libxt-dev x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 25 newly installed, 0 to remove and 49 not upgraded.
Need t

Stable-Baselines3 works on environments that follow the [gym interface](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html).
You can find a list of available environment [here](https://gymnasium.farama.org/environments/classic_control/).

Not all algorithms can work with all action spaces, you can find more in this [recap table](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html)

In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3.common.env_checker import check_env


#Enviorment declaration




In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def seasonal_duck_curve(hour, season, noise_scale=0.03):
    A = 400  # Baseline demand

    # Initialize variables with default values to avoid UnboundLocalError
    B, t_morning, k_morning = 0, 0, 0
    C, t_dip, k_dip = 0, 0, 0
    D, mu_dip, sigma_dip = 0, 0, 0
    E, t_evening, k_evening = 0, 0, 0
    F, t_early, k_early = 0, 0, 0

    if season == 1: # summer
        B, t_morning, k_morning = 100, 7, 1  # Morning rise (lower due to AC usage later)
        C, t_dip, k_dip = 80, 12, 1  # Midday dip (solar production)
        D, mu_dip, sigma_dip = 120, 14, 2  # Midday Gaussian dip (solar energy impact)
        E, t_evening, k_evening = 250, 18, 1  # Higher evening peak (AC usage)
        F, t_early, k_early = 30, 4, 0.8  # Early rise (moderate)

    elif season == 2: #winter
        B, t_morning, k_morning = 180, 6, 1  # Sharper morning rise (heating demand)
        C, t_dip, k_dip = 40, 12, 1  # Smaller midday dip (less solar impact)
        D, mu_dip, sigma_dip = 80, 14, 2  # Smaller midday Gaussian dip
        E, t_evening, k_evening = 220, 17, 1  # High evening peak (heating demand)
        F, t_early, k_early = 60, 4, 1  # Strong early morning rise

    elif season == 3: # "spring" or season == "autumn":
        B, t_morning, k_morning = 130, 7, 1  # Moderate morning rise
        C, t_dip, k_dip = 90, 12, 1  # Midday dip (moderate solar impact)
        D, mu_dip, sigma_dip = 100, 14, 2  # Balanced midday dip
        E, t_evening, k_evening = 180, 18, 1  # Moderate evening peak
        F, t_early, k_early = 40, 4, 0.8  # Moderate early rise

    early_rise = F / (1 + np.exp(-k_early * (hour - t_early)))
    morning_rise = B / (1 + np.exp(-k_morning * (hour - t_morning)))
    midday_smooth_dip = -C / (1 + np.exp(-k_dip * (hour - t_dip)))
    midday_gaussian_dip = -D * np.exp(-((hour - mu_dip)**2) / (2 * sigma_dip**2))
    evening_ramp = E / (1 + np.exp(-k_evening * (hour - t_evening)))

    demand = A + early_rise + morning_rise + midday_smooth_dip + midday_gaussian_dip + evening_ramp

    # # Add random noise to the demand (Gaussian noise with mean=0 and standard deviation=noise_std)
    # noise = np.random.normal(0, noise_std, size=demand.shape)
    #I am changing the gaussian noise to a uniform noise which is +- 3% of the demand
    noise = np.random.uniform(-noise_scale * demand, noise_scale * demand)
    return demand + noise

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def electricity_price_function(t, season, demand_factor ,noise_scale=0.03):
    season_params = {
        1: {"A": 30, "B": 15, "C": 10},  # Summer
        2: {"A": 28, "B": 14, "C": 9},   # Winter
        3: {"A": 25, "B": 12, "C": 8}    # Spring/Autumn
    }

    A_q = season_params[season]["A"]
    B_q = season_params[season]["B"]
    C_q = season_params[season]["C"]

    base_price = A_q + B_q * np.cos(2 * np.pi * t / 24) + C_q * np.cos(4 * np.pi * t / 24)

    # Adjust price based on demand factor
    noise = np.random.uniform(-noise_scale * base_price, noise_scale * base_price)
    adjusted_price = base_price + noise
    #noise = np.random.normal(0, 2)
    return max(adjusted_price , 0) # + noise

In [ ]:
DEFAULT_MAX_TIME_STEPS = 365 #year
INITIAL_BATTERY_CAPACITY = 4200 # in kilow wat

from gymnasium.spaces import Box, Discrete
from numpy import float32
import random
from sklearn.preprocessing import MinMaxScaler


MIN_DEMAND = 194
MAX_DEMAND = 824
MIN_ELECT_PRICE = 4.85
MAX_ELECT_PRICE = 56.65
MIN_BAT_PRICE = 36743
MAX_BAT_PRICE = 254378
MIN_REWARD = -254378
MAX_REWARD = 226939.9
class ElectricityMarketEnv(gym.Env):
    """
    Custom Environment for the electricity market using Gymnasium.
    """

    def __init__(self, max_timesteps=DEFAULT_MAX_TIME_STEPS, season=0):
        super(ElectricityMarketEnv, self).__init__()
        self.timestep = 1
        self.season = season
        self.season = self.get_season()
        # Environment parameters

        self.battery_time_bought = 0
        self.step_in_season = 0

        self.max_timesteps = max_timesteps + 1 #episode length in days

        self.battery_percentage = 100
        self.demand, self.elect_price = self.__get_new_demand_and_elect_price()
        self.capacity = 100
        self.min_max_scaler_demand = MinMaxScaler().fit(np.array([[MIN_DEMAND], [MAX_DEMAND]])) #found it
        self.min_max_scaler_percentage = MinMaxScaler().fit(np.array([[0], [100]])) #found it
        self.min_max_scaler_elect_price = MinMaxScaler().fit(np.array([[MIN_ELECT_PRICE], [MAX_ELECT_PRICE]])) #found it
        self.min_max_scaler_bat_price = MinMaxScaler().fit(np.array([[MIN_BAT_PRICE], [MAX_BAT_PRICE]])) # need to find
        self.reward_normalizer = MinMaxScaler().fit(np.array([[MIN_REWARD], [MAX_REWARD]]))
        # Action space: Continuous range [-capacity, capacity]
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)

        # Observation space: [SoC, Dt, Pt]
        self.observation_space = spaces.Box(
            low=np.array([0, 0, 0, 0, 0], dtype=np.float32),  # Min values for SoC, Dt, Pt, capacity
            high=np.array([100, np.inf, np.inf, np.inf, 100], dtype=np.float32),  # Max values for SoC, Dt, Pt
            dtype=np.float32,
        )

        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.timestep = 1
        self.season = self.get_season()
        # Environment parameters

        self.battery_time_bought = 0
        self.step_in_season = 0
        self.battery_percentage = 100
        state = self._get_state()
        return state, {}

    def step(self, action):
        requested_charge_in_percentage, to_buy_battery = action
        requested_charge_in_percentage = 100 * requested_charge_in_percentage
        reward = 0

        if to_buy_battery > 0:
          self.battery_time_bought = self.timestep
          self.capacity = 100
          self.battery_percentage = 100 #new battery is charged
          reward = -self.bat_price
          # Update timestep
          self.timestep += 1
          done = self.timestep >= self.max_timesteps  # Episode ends after max timesteps
          # Return next state, reward, and done flag
          next_state = self._get_state()
          return next_state, reward, done, False, {}

        if requested_charge_in_percentage >0:
            self.battery_percentage += requested_charge_in_percentage
            self.battery_percentage = min(100, self.battery_percentage)
        else:
          percentage_discharged = min(self.battery_percentage, -requested_charge_in_percentage)
          self.battery_percentage -= percentage_discharged
          power_discharged = INITIAL_BATTERY_CAPACITY * self.capacity * (percentage_discharged / 100)
          power_to_sell = max(0, power_discharged - self.demand)
          reward += power_to_sell * self.elect_price

        # Update timestep
        self.timestep += 1
        done = self.timestep >= self.max_timesteps  # Episode ends after max timesteps
        # Return next state, reward, and done flag
        next_state = self._get_state()
        return next_state, self.reward_normalizer.transform([[reward]])[0][0], done, False, {}

    def _get_state(self):
        """
        Return the current state as [SoC, Dt, Pt].
        """
        self.season = self.get_season()
        self.demand, self.elect_price = self.__get_new_demand_and_elect_price()
        self.capacity = self.__get_battery_capacity(self.timestep - self.battery_time_bought)# percentage from new battery
        self.bat_price = self.__get_battery_price(self.elect_price) #dollars
        return np.array([
            self.min_max_scaler_percentage.transform([[float(self.battery_percentage)]])[0][0],
            self.min_max_scaler_demand.transform([[float(self.demand)]])[0][0],
            self.min_max_scaler_elect_price.transform([[float(self.elect_price)]])[0][0],
            self.min_max_scaler_bat_price.transform([[float(self.bat_price)]])[0][0],
            self.min_max_scaler_percentage.transform([[float(self.capacity)]])[0][0]
        ], dtype=np.float32)

    def get_season(self):
      if self.season != 0:
        self.step_in_season = self.timestep
        return self.season
      if 182 <= self.timestep <= 243:
        self.step_in_season = self.timestep - 182
        return 1  # Summer (July-August)
      elif 335 <= self.timestep or self.timestep <= 59:
        return 2  # Winter (December-February)
      else:
        return 3  # Spring/Autumn (March-June, Sept-Nov)

    def __get_new_demand_and_elect_price(self):
        """
        Simulate electricity demand as a periodic function with noise.
        The Demand has to be non negative >= 0
        Question: how frequent the demand changes? if i return the state then when the action is done should it contain the same demand/price as returned.
        """
        hour = random.randint(0, 23)
        demand = seasonal_duck_curve(hour, self.season)
        return demand, electricity_price_function(self.step_in_season, self.season, demand)

    def __get_battery_price(self, electricity_price, base_price=16373, scaling_factor=1):
      return base_price +   INITIAL_BATTERY_CAPACITY * electricity_price * scaling_factor

    def __get_battery_capacity(self, time, C0=100, kc=0.0059, kn=0.01, cycles_per_day=1):
      """
      Computes the remaining battery capacity as a function of time (t).

      Parameters:
          t (float): Time in days.
          C0 (float): Initial battery capacity percentage (default: 100%).
          kc (float): Calendar aging constant per day.
          kn (float): Cycle aging constant per cycle.
          cycles_per_day (float): Number of charge/discharge cycles per day.

      Returns:
          float: Remaining battery capacity percentage.
      """
      n = cycles_per_day * time  # Total number of cycles
      return C0 * np.exp(-(kc * time + kn * n))

In [ ]:
env = ElectricityMarketEnv()
check_env(env)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Lets set a constant seed =  22

In [ ]:
SEED = 22

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Evaluation Functions


In [ ]:

def get_rewards(
    model,
    num_episodes: int = 100,
    deterministic: bool = True,
) -> float:
    # This function will only work for a single environment
    vec_env = model.get_env()
    obs = vec_env.reset()
    all_episode_rewards = []
    for _ in range(num_episodes):
        episode_rewards = []
        done = False
        while not done:
            # _states are only useful when using LSTM policies
            # `deterministic` is to use deterministic actions
            action, _states = model.predict(obs, deterministic=deterministic)
            # here, action, rewards and dones are arrays
            # because we are using vectorized env
            obs, reward, done, _info = vec_env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))
    return all_episode_rewards

In [ ]:
def evaluate(
    model,
    num_episodes: int = 100,
    deterministic: bool = True,
) -> float:
  all_episode_rewards = get_rewards(model, num_episodes, deterministic)
  mean_episode_reward = np.mean(all_episode_rewards)
  print(f"Mean reward: {mean_episode_reward:.2f} - Num episodes: {num_episodes}")


In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy


In [ ]:
%pip install tensorboard
%load_ext tensorboard


In [ ]:
import time
import numpy as np
import tensorflow as tf
from stable_baselines3.common.callbacks import BaseCallback

class RLComparisonCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(RLComparisonCallback, self).__init__(verbose)
        self.start_time = time.time()

    def _on_training_start(self) -> None:
        self.episode_rewards = []
        self.episode_lengths = []
        self.total_timesteps = 0

    def _on_step(self) -> bool:
        if self.locals.get("dones") is not None:
            for done, reward, info in zip(self.locals["dones"], self.locals["rewards"], self.locals["infos"]):
                if done:
                    episode_reward = info.get("episode", {}).get("r", reward)
                    episode_length = info.get("episode", {}).get("l", 0)

                    self.episode_rewards.append(episode_reward)
                    self.episode_lengths.append(episode_length)
                    self.total_timesteps += episode_length

                    # Logging metrics after each episode using log scale
                    avg_reward = np.mean(self.episode_rewards[-100:])
                    cum_reward = np.sum(self.episode_rewards)
                    discounted_reward = np.sum([r * (0.99 ** i) for i, r in enumerate(self.episode_rewards)])
                    convergence_rate = np.std(self.episode_rewards[-100:])
                    sample_efficiency = cum_reward / max(1, self.total_timesteps)
                    stability = np.std(self.episode_rewards)
                    policy_entropy = info.get('entropy', 0)
                    time_complexity = time.time() - self.start_time
                    space_complexity = self.model.policy.parameters_to_vector().nbytes

                    self.logger.record("custom/log_average_reward", np.log1p(avg_reward))
                    self.logger.record("custom/log_cumulative_reward", np.log1p(cum_reward))
                    self.logger.record("custom/log_discounted_reward", np.log1p(discounted_reward))
                    self.logger.record("custom/log_convergence_rate", np.log1p(convergence_rate))
                    self.logger.record("custom/log_sample_efficiency", np.log1p(sample_efficiency))
                    self.logger.record("custom/log_stability", np.log1p(stability))
                    self.logger.record("custom/log_policy_entropy", np.log1p(policy_entropy))
                    self.logger.record("custom/log_time_complexity", np.log1p(time_complexity))
                    self.logger.record("custom/log_space_complexity", np.log1p(space_complexity))

        return True


#Random Baseline

In [ ]:
obs = env.reset(seed=SEED)
n_steps = 60
for _ in range(n_steps):
    # Random action
    action = env.action_space.sample()
    obs, reward, done, _, info = env.step(action)
    if done:
        obs = env.reset(seed=SEED)

In [ ]:
reward

-114052.85400197796

In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback, CallbackList
checkpoint_callback = CheckpointCallback(
    save_freq=50000,  # Save every 100,000 steps
    save_path='./models/',  # Directory to save the model
    name_prefix='ddpg_model'  # Prefix for the saved model files
)
callback = CallbackList([RLComparisonCallback(), checkpoint_callback])


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir ./tensorboard/

<IPython.core.display.Javascript object>

#Stacked combine version

In [ ]:
import gym
import numpy as np
from stable_baselines3 import PPO, TD3, DDPG, A2C
from sklearn.linear_model import LogisticRegression

# Hyperparameters
NUM_EPISODES = 100

# Create environment
env = gym.make('CartPole-v1')  # Replace with your custom environment

# Train base models
ppo_model = PPO('MlpPolicy', env, verbose=0)
td3_model = TD3('MlpPolicy', env, verbose=0)
ddpg_model = DDPG('MlpPolicy', env, verbose=0)
a2c_model = A2C('MlpPolicy', env, verbose=0)

ppo_model.learn(total_timesteps=10000)
td3_model.learn(total_timesteps=10000)
ddpg_model.learn(total_timesteps=10000)
a2c_model.learn(total_timesteps=10000)

# Stacking algorithm (Meta-Learner with Logistic Regression)
class StackingAgent:
    def __init__(self, models):
        self.models = models
        self.meta_learner = LogisticRegression()

    def collect_training_data(self, env, num_episodes=50):
        X, y = [], []
        for _ in range(num_episodes):
            obs = env.reset()
            done = False
            while not done:
                model_actions = [model.predict(obs, deterministic=True)[0] for model in self.models]
                X.append(model_actions)
                action = np.random.choice(model_actions)
                obs, reward, done, _ = env.step(action)
                y.append(action)
        return np.array(X), np.array(y)

    def train_meta_learner(self, env):
        X, y = self.collect_training_data(env)
        self.meta_learner.fit(X, y)

    def predict(self, obs):
        model_actions = [model.predict(obs, deterministic=True)[0] for model in self.models]
        action = self.meta_learner.predict([model_actions])[0]
        return action

# Combine models with stacking
stacked_agent = StackingAgent(models=[ppo_model, td3_model, ddpg_model, a2c_model])
stacked_agent.train_meta_learner(env)

# Evaluate stacked agent
for episode in range(NUM_EPISODES):
    obs = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = stacked_agent.predict(obs)
        obs, reward, done, _ = env.step(action)
        total_reward += reward

    print(f"Episode {episode + 1}: Total Reward = {total_reward}")